Подключение необходимых модулей и загрузка выборки

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
os.chdir('/content/drive/MyDrive/Colab Notebooks/NIR')

In [ ]:
df=pd.read_csv("data_capec_multilabel_ishodn.csv", sep=',', low_memory=False)

In [ ]:
display(df)

,timestamp,src_ip,src_port,dst_ip,dst_port,request_http_method,request_http_request,request_http_protocol,request_user_agent,request_referer,...,126 - Path Traversal,66 - SQL Injection,16 - Dictionary-based Password Attack,310 - Scanning for Vulnerable Software,153 - Input Data Manipulation,248 - Command Injection,274 - HTTP Verb Tampering,194 - Fake the Source of Data,34 - HTTP Response Splitting,33 - HTTP Request Smuggling
0,17/Jul/2020:12:23:34 +0100,172.26.0.1,55894.0,172.26.0.4,80.0,GET,/,HTTP/1.1,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3...,NaN,...,0,0,0,0,0,0,0,0,0,0
1,17/Jul/2020:12:23:41 +0100,172.26.0.1,55897.0,172.26.0.4,80.0,GET,/blog/index.php/2020/04/04/voluptatum-reprehen...,HTTP/1.1,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; r...,NaN,...,0,0,0,0,0,0,0,0,0,0
2,17/Jul/2020:12:23:51 +0100,172.26.0.1,55901.0,172.26.0.4,80.0,GET,/blog/xmlrpc.php?rsd,HTTP/1.1,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT ...,NaN,...,0,0,0,0,0,0,0,0,0,0
3,17/Jul/2020:12:23:51 +0100,172.26.0.1,55902.0,172.26.0.4,80.0,GET,/,HTTP/1.1,Mozilla/5.0 (Linux; Android 4.2.2; SM-T110 Bui...,NaN,...,0,0,0,0,0,0,0,0,0,0
4,17/Jul/2020:12:23:58 +0100,172.26.0.1,55903.0,172.26.0.4,80.0,GET,/blog/index.php/2020/04/04/nihil-tenetur-et-ad...,HTTP/1.1,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2...,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907810,29/Jul/2020:22:26:31 +0100,172.26.0.1,53171.0,172.26.0.4,80.0,GET,/,HTTP/1.1,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5)...,NaN,...,0,0,0,0,0,0,0,0,0,0
907811,29/Jul/2020:22:27:26 +0100,172.26.0.1,53179.0,172.26.0.4,80.0,GET,/,HTTP/1.1,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_5)...,NaN,...,0,0,0,0,0,0,0,0,0,0
907812,29/Jul/2020:22:27:49 +0100,172.26.0.1,53185.0,172.26.0.4,80.0,GET,/,HTTP/1.1,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,NaN,...,0,0,0,0,0,0,0,0,0,0
907813,29/Jul/2020:22:28:28 +0100,172.26.0.1,53192.0,172.26.0.4,80.0,GET,/,HTTP/1.1,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,NaN,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
display(df['response_http_status_message'])

0         OK
1         OK
2         OK
3         OK
4         OK
          ..
907810    OK
907811    OK
907812    OK
907813    OK
907814    OK
Name: response_http_status_message, Length: 907815, dtype: object

In [ ]:
df.shape

(907815, 38)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907815 entries, 0 to 907814
Data columns (total 38 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   timestamp                               907813 non-null  object 
 1   src_ip                                  907813 non-null  object 
 2   src_port                                907813 non-null  float64
 3   dst_ip                                  907813 non-null  object 
 4   dst_port                                907813 non-null  float64
 5   request_http_method                     907813 non-null  object 
 6   request_http_request                    907813 non-null  object 
 7   request_http_protocol                   907813 non-null  object 
 8   request_user_agent                      903981 non-null  object 
 9   request_referer                         291394 non-null  object 
 10  request_host                            9077

In [ ]:
df.dtypes

timestamp                                  object
src_ip                                     object
src_port                                  float64
dst_ip                                     object
dst_port                                  float64
request_http_method                        object
request_http_request                       object
request_http_protocol                      object
request_user_agent                         object
request_referer                            object
request_host                               object
request_origin                             object
request_cookie                             object
request_content_type                       object
request_accept                             object
request_accept_language                    object
request_accept_encoding                    object
request_do_not_track                      float64
request_connection                         object
request_body                               object


In [ ]:
df.pop('248 - Command Injection')

0         0
1         0
2         0
3         0
4         0
         ..
907810    0
907811    0
907812    0
907813    0
907814    0
Name: 248 - Command Injection, Length: 907815, dtype: int64

In [ ]:
df.dtypes

timestamp                                  object
src_ip                                     object
src_port                                  float64
dst_ip                                     object
dst_port                                  float64
request_http_method                        object
request_http_request                       object
request_http_protocol                      object
request_user_agent                         object
request_referer                            object
request_host                               object
request_origin                             object
request_cookie                             object
request_content_type                       object
request_accept                             object
request_accept_language                    object
request_accept_encoding                    object
request_do_not_track                      float64
request_connection                         object
request_body                               object


Предварительная оцифровка данных полей веб-запроса

In [ ]:
spis_column = list(df.columns)
for i in range(df.shape[0]):
  for j in spis_column[:24]:
    value = df[j][i]
    if pd.isnull(value) == True:
      value = 0
      df.at[i, j] = float(value)
      #print(value)
    else:
      value_1 = str(value).lower()
      L = len(value_1)
      sum = 0
      for symb in value_1:
        symb_ascii = ord(symb)
        sum = sum + symb_ascii
      Decimal_value = sum/L
      df.at[i, j] = Decimal_value
      #print(Decimal_value)

In [ ]:
display(df)

,timestamp,src_ip,src_port,dst_ip,dst_port,request_http_method,request_http_request,request_http_protocol,request_user_agent,request_referer,...,88 - OS Command Injection,126 - Path Traversal,66 - SQL Injection,16 - Dictionary-based Password Attack,310 - Scanning for Vulnerable Software,153 - Input Data Manipulation,274 - HTTP Verb Tampering,194 - Fake the Source of Data,34 - HTTP Response Splitting,33 - HTTP Request Smuggling
0,56.538462,49.3,52.142857,49.6,49.5,106.666667,47.0,79.875,78.719008,0.0,...,0,0,0,0,0,0,0,0,0,0
1,56.461538,49.3,52.571429,49.6,49.5,106.666667,91.519481,79.875,77.061728,0.0,...,0,0,0,0,0,0,0,0,0,0
2,56.5,49.3,50.571429,49.6,49.5,106.666667,97.1,79.875,73.961783,0.0,...,0,0,0,0,0,0,0,0,0,0
3,56.5,49.3,50.714286,49.6,49.5,106.666667,47.0,79.875,78.132812,0.0,...,0,0,0,0,0,0,0,0,0,0
4,56.769231,49.3,50.857143,49.6,49.5,106.666667,85.793103,79.875,78.743802,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907810,56.692308,49.3,50.142857,49.6,49.5,106.666667,47.0,79.875,79.108333,0.0,...,0,0,0,0,0,0,0,0,0,0
907811,56.884615,49.3,51.285714,49.6,49.5,106.666667,47.0,79.875,79.075,0.0,...,0,0,0,0,0,0,0,0,0,0
907812,57.076923,49.3,50.857143,49.6,49.5,106.666667,47.0,79.875,77.098765,0.0,...,0,0,0,0,0,0,0,0,0,0
907813,57.0,49.3,50.571429,49.6,49.5,106.666667,47.0,79.875,77.889908,0.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for k in spis_column[:24]:
  df[k] = df[k].astype('float', copy = False)

In [ ]:
display(df)

,timestamp,src_ip,src_port,dst_ip,dst_port,request_http_method,request_http_request,request_http_protocol,request_user_agent,request_referer,...,88 - OS Command Injection,126 - Path Traversal,66 - SQL Injection,16 - Dictionary-based Password Attack,310 - Scanning for Vulnerable Software,153 - Input Data Manipulation,274 - HTTP Verb Tampering,194 - Fake the Source of Data,34 - HTTP Response Splitting,33 - HTTP Request Smuggling
0,56.538462,49.3,52.142857,49.6,49.5,106.666667,47.000000,79.875,78.719008,0.0,...,0,0,0,0,0,0,0,0,0,0
1,56.461538,49.3,52.571429,49.6,49.5,106.666667,91.519481,79.875,77.061728,0.0,...,0,0,0,0,0,0,0,0,0,0
2,56.500000,49.3,50.571429,49.6,49.5,106.666667,97.100000,79.875,73.961783,0.0,...,0,0,0,0,0,0,0,0,0,0
3,56.500000,49.3,50.714286,49.6,49.5,106.666667,47.000000,79.875,78.132812,0.0,...,0,0,0,0,0,0,0,0,0,0
4,56.769231,49.3,50.857143,49.6,49.5,106.666667,85.793103,79.875,78.743802,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907810,56.692308,49.3,50.142857,49.6,49.5,106.666667,47.000000,79.875,79.108333,0.0,...,0,0,0,0,0,0,0,0,0,0
907811,56.884615,49.3,51.285714,49.6,49.5,106.666667,47.000000,79.875,79.075000,0.0,...,0,0,0,0,0,0,0,0,0,0
907812,57.076923,49.3,50.857143,49.6,49.5,106.666667,47.000000,79.875,77.098765,0.0,...,0,0,0,0,0,0,0,0,0,0
907813,57.000000,49.3,50.571429,49.6,49.5,106.666667,47.000000,79.875,77.889908,0.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.dtypes

timestamp                                 float64
src_ip                                    float64
src_port                                  float64
dst_ip                                    float64
dst_port                                  float64
request_http_method                       float64
request_http_request                      float64
request_http_protocol                     float64
request_user_agent                        float64
request_referer                           float64
request_host                              float64
request_origin                            float64
request_cookie                            float64
request_content_type                      float64
request_accept                            float64
request_accept_language                   float64
request_accept_encoding                   float64
request_do_not_track                      float64
request_connection                        float64
request_body                              float64


Разделение набора данных на обучающую и тестовую, обучение классификаторов

In [ ]:
X = df[spis_column[:24]]
Y = df[spis_column[24:]]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.35, random_state = 42)

In [ ]:
print('Shape of X train:' , X_train.shape)
print('Shape of y train:' , Y_train.shape)
print('Shape of X test:', X_test.shape)
print('Shape of Y test:', Y_test.shape)

Shape of X train: (453907, 24)
Shape of y train: (453907, 13)
Shape of X test: (453908, 24)
Shape of Y test: (453908, 13)


In [ ]:
targ_names = list(df.columns)[24:]
print(targ_names)

['000 - Normal', '272 - Protocol Manipulation', '242 - Code Injection', '88 - OS Command Injection', '126 - Path Traversal', '66 - SQL Injection', '16 - Dictionary-based Password Attack', '310 - Scanning for Vulnerable Software', '153 - Input Data Manipulation', '274 - HTTP Verb Tampering', '194 - Fake the Source of Data', '34 - HTTP Response Splitting', '33 - HTTP Request Smuggling']


In [ ]:
# Стандартизация данных
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.fit_transform(X_test)

In [ ]:
# Модель Random Forest
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train_scaled, Y_train)
y_pred = clf.predict(X_test_scaled)

In [ ]:
from sklearn. metrics import classification_report
print(classification_report(Y_test, y_pred, target_names = targ_names))

                                        precision    recall  f1-score   support

                          000 - Normal       0.94      0.92      0.93    184124
           272 - Protocol Manipulation       0.93      0.52      0.66      3135
                  242 - Code Injection       0.94      0.70      0.80      5552
             88 - OS Command Injection       0.95      0.35      0.51      2565
                  126 - Path Traversal       0.91      0.76      0.83      7190
                    66 - SQL Injection       0.87      0.85      0.86     87743
 16 - Dictionary-based Password Attack       1.00      0.86      0.92       649
310 - Scanning for Vulnerable Software       1.00      1.00      1.00       929
         153 - Input Data Manipulation       0.16      0.97      0.28       775
             274 - HTTP Verb Tampering       1.00      0.99      0.99      1910
         194 - Fake the Source of Data       0.80      0.42      0.55     19528
          34 - HTTP Response Splitting 

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print("Accuracy:", metrics.accuracy_score(Y_test, y_pred))

Accuracy: 0.8426398015962938


In [ ]:
details = df.apply(lambda x : True
                   if x['000 - Normal'] == 1 else False, axis = 1)

# Count number of True in the series
num_rows = len(details[details == True].index)
print(num_rows)

525195


In [ ]:
# Модель XGBoost
xgb_estimator = xgb.XGBClassifier()
multilabel_model = MultiOutputClassifier(xgb_estimator)
multilabel_model.fit(X_train_scaled, Y_train)
xgb_clf_preds = multilabel_model.predict(X_test_scaled)

In [ ]:
print("Accuracy:", metrics.accuracy_score(Y_test, xgb_clf_preds))

Accuracy: 0.702866530704736


In [ ]:
print(classification_report(Y_test, xgb_clf_preds, target_names = targ_names))

                                        precision    recall  f1-score   support

                          000 - Normal       0.93      0.83      0.88    184124
           272 - Protocol Manipulation       0.96      0.24      0.38      3135
                  242 - Code Injection       0.88      0.31      0.45      5552
             88 - OS Command Injection       0.96      0.01      0.02      2565
                  126 - Path Traversal       0.83      0.32      0.46      7190
                    66 - SQL Injection       0.87      0.70      0.77     87743
 16 - Dictionary-based Password Attack       0.98      0.42      0.59       649
310 - Scanning for Vulnerable Software       1.00      0.90      0.95       929
         153 - Input Data Manipulation       0.10      0.97      0.19       775
             274 - HTTP Verb Tampering       1.00      1.00      1.00      1910
         194 - Fake the Source of Data       0.76      0.26      0.39     19528
          34 - HTTP Response Splitting 

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
